In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

In [2]:
!mkdir -p data results logs


In [5]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata


In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Todos:

 

1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






Run following in local Jupyter

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


Run following 2 cells for colab to import libraries

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


Run following if the library code is modified locally

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
model = "gstock-4a-test"
tickers = ['ABB', 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY', 'DPSGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA', 
            'OZON', 'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NXPI', 'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'VWAPY', 'YARIY', 'ZURVY']


In [7]:
findata.EPOCHS=200
model = "val-4b-test"
tickers = ['AAL', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BGFV', 'BXC',
            'CBT', 'CC', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI',
            'CG',  'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DVN', 'EMN', 
            'F', 'FCX', 'FIS', 'FLO', 'FMS', 'GLW', 'GWW',
            'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN', 'KD', 'KHC', 'KMB', 
             'M', 'MMM', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK', 
            'PARA', 'PCAR', 'PETS', 'PNR', 'PSTL', 'PXD', 'RCL', 'RH',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SWK',
            'T', 'TCN', 'TGT', 'TDG', 'TWI', 'UAL', 'UBA', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [ ]:
findata.EPOCHS=200
model = "cry-4b-test"
tickers = [ 'ADA-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'XRP-USD']

In [ ]:
findata.EPOCHS=200
model = "etf-5a1-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX', 
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM', 
            'ONLN', 'QQQ', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
model = "mcap-5a-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']

In [ ]:
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [9]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
7,BGFV,1.04,0.91,160.90,132.89,7.70,5.030000,-0.35,0.80,219.49,177.30,7.600000,-0.01,0.68,75.58,68.47,4.700000,-0.39,-0.35,8.842857
35,IPI,8.19,0.88,499.26,373.62,18.93,12.960000,-0.32,0.84,326.69,353.32,19.400000,0.02,0.74,235.28,526.30,12.410000,-0.34,-0.32,8.425000
15,CIVI,8.87,0.94,474.16,307.83,69.95,57.500000,-0.18,0.77,462.01,294.38,70.570000,0.01,0.64,312.92,134.93,57.779999,-0.17,-0.17,9.047059
3,ARCH,20.90,0.89,766.03,642.23,110.16,91.790001,-0.17,0.81,1071.92,811.65,120.199997,0.09,0.66,719.32,404.76,84.720001,-0.23,-0.17,8.252941
26,FCX,5.85,0.89,264.65,216.89,34.77,28.780001,-0.17,0.74,232.84,162.25,41.930000,0.21,0.70,210.61,203.21,28.049999,-0.19,-0.17,8.252941
10,CC,6.97,0.91,246.06,126.37,28.52,24.209999,-0.15,0.79,233.55,154.56,30.559999,0.07,0.73,244.82,235.55,23.030001,-0.19,-0.15,8.500000
8,BXC,5.34,0.91,699.66,429.15,84.15,71.889999,-0.15,0.84,700.51,464.83,97.620003,0.16,0.69,426.86,441.45,62.660000,-0.26,-0.15,8.500000
16,CG,15.94,0.94,210.88,223.71,27.95,24.160000,-0.14,0.76,198.77,181.25,29.459999,0.05,0.63,163.44,189.50,23.160000,-0.17,-0.14,8.971429
56,SAFE,27.98,0.81,468.75,527.66,24.50,28.450001,0.16,0.76,424.55,464.70,29.540001,0.21,0.65,302.22,550.22,26.780001,0.09,0.16,6.912500
20,CSL,99.37,0.91,1119.69,964.58,216.83,188.940002,-0.13,0.82,1265.49,763.05,211.619995,-0.02,0.67,827.18,708.67,179.820007,-0.17,-0.13,8.407692


In [ ]:
findata.EPOCHS=200
model = "cetf-5b-test"
tickers = ['ARGT', 'EWD', 'CUBS', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']

In [10]:
findata.EPOCHS=200
model = "com-5b-test"
tickers = [ 'GLD', 'ISHG', 'JJU', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [11]:
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

Epoch 1/200
127/129 [============================>.] - ETA: 0s - loss: 0.0070 - mean_absolute_error: 0.0806
Epoch 1: val_loss improved from inf to 0.00313, saving model to results/com-5b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
129/129 [==============================] - 8s 19ms/step - loss: 0.0069 - mean_absolute_error: 0.0805 - val_loss: 0.0031 - val_mean_absolute_error: 0.0586
Epoch 2/200
128/129 [============================>.] - ETA: 0s - loss: 0.0040 - mean_absolute_error: 0.0666
Epoch 2: val_loss improved from 0.00313 to 0.00296, saving model to results/com-5b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
129/129 [==============================] - 1s 10ms/step - loss: 0.0040 - mean_absolute_error: 0.0666 - val_loss: 0.0030 - val_mean_absolute_error: 0.0570
Epoch 3/200
127/129 [=========================

In [12]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
3,PDBC,6.64,0.85,49.31,29.89,13.70,13.150000,-0.04,0.67,33.83,13.22,14.170000,0.03,0.51,14.50,0.59,13.190000,-0.04,-0.04,4.750000
4,SLV,11.58,0.91,109.75,109.69,21.38,22.280001,0.04,0.67,59.28,47.66,22.049999,0.03,0.49,-2.42,-31.83,20.600000,-0.04,0.03,6.100000
1,ISHG,63.65,0.87,95.02,100.16,68.58,70.459999,0.03,0.54,2.27,8.19,70.470001,0.03,0.49,-13.03,14.48,68.519997,-0.00,0.03,4.366667
5,SOYB,13.64,0.84,79.56,37.36,24.92,24.219999,-0.03,0.66,62.22,13.14,25.299999,0.02,0.53,21.12,-16.49,24.059999,-0.03,-0.03,3.066667
2,JJU,33.17,0.83,203.95,189.19,46.15,44.630001,-0.03,0.61,65.70,40.09,46.599998,0.01,0.51,52.93,33.55,43.990002,-0.05,-0.03,2.633333
10,WEAT,5.02,0.83,31.51,31.18,6.26,6.070000,-0.03,0.63,13.90,6.38,6.490000,0.04,0.60,4.36,6.31,5.850000,-0.07,-0.03,2.633333
8,UUP,24.03,0.87,33.45,28.70,28.59,28.139999,-0.02,0.61,16.83,11.94,28.820000,0.01,0.53,5.55,2.28,28.129999,-0.02,-0.02,2.200000
0,GLD,126.65,0.89,542.16,342.78,180.92,181.300003,0.00,0.68,333.04,151.85,184.889999,0.02,0.49,90.52,-15.68,176.960007,-0.02,0.00,0.000000
6,TIP,96.70,0.86,116.58,95.48,107.73,106.529999,-0.01,0.64,72.10,41.05,108.639999,0.01,0.43,5.33,-26.01,106.639999,-0.01,-0.01,0.000000
7,TLT,92.42,0.84,217.48,373.53,101.09,101.519997,0.00,0.56,79.31,187.82,105.029999,0.04,0.60,3.53,170.68,100.470001,-0.01,0.00,0.000000


In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV', 
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE', 
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE', 
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [13]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 

sending incremental file list
com-5b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
com-5b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
com-5b-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,645,646 bytes  received 73 bytes  19,291,438.00 bytes/sec
total size is 134,922,688  speedup is 13.99
